In [120]:
from mtcnn import MTCNN
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np

In [121]:
def load_face(filename, required_size=(160, 160)):
    # carregando imagem de arquivo
    image = Image.open(filename)
    
    #converter para RGB
    image = image.convert('RGB')
    
    return asarray(image)

In [122]:
# Carregando imagens de face de um diretório
def load_faces(directory_src):
    
    faces = list()
    
    #iterando arquivos
    for filename in listdir(directory_src):
    
        path = directory_src + filename

        try:
            faces.append(load_face(path))
        except:
            print("Erro na imagem {}".format(path))
        
    return faces
    

In [123]:
## Carregando todo o dataset de imagens de faces

def load_fotos(directory_src):
    
    X, y = list(), list()
    
    # iterar pastas por classes
    
    for subdir in listdir(directory_src):
        
        #path
        path = directory_src + subdir + '/'
        
        if not isdir(path):
            continue
            
        faces = load_faces(path)
        
        labels = [subdir for _ in range(len(faces))]
        
        # sumarizar progresso
        print('>Carregadas %d faces da classe: %s' % (len(faces), subdir))
        
        X.extend(faces)
        y.extend(labels)
        
    return asarray(X), asarray(y)

## CARREGANDO TODAS AS IMAGENS

In [124]:
trainX, trainy = load_fotos(directory_src="/home/charles/Imagens/face-recognition/validation/faces/")

>Carregadas 12 faces da classe: poliana
>Carregadas 14 faces da classe: charles


In [125]:
trainX.shape

(26, 160, 160, 3)

In [126]:
trainy.shape

(26,)

In [106]:
from tensorflow.keras.models import load_model

In [107]:
model = load_model('facenet_keras.h5')

In [108]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

## **FUNÇÃO GERADORA DE EMBEDDINGS**

In [127]:
def get_embedding(model, face_pixels):
    
    #PADROZINAÇÃO
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    
    #TRANSFORMAR A FACE EM 1 ÚNICO EXEMPLO
    
    #(160,160) -> (1,160,160)
    
    samples = np.expand_dims(face_pixels, axis=0)
    
    #REALIZAR A PREDIÇÃO GERANDO O EMBEDDING
    yhat = model.predict(samples)
    
    ## [[1,2..128]]
    
    return yhat[0]

In [128]:
newTrainX = list()

In [129]:
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)

In [130]:
newTrainX = asarray(newTrainX)

In [131]:
newTrainX.shape

(26, 128)

In [132]:
import pandas as pd

In [133]:
df = pd.DataFrame(data=newTrainX)

In [134]:
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-2.179760,0.270186,-2.484582,0.536236,-0.870110,-0.704696,-0.836423,0.391036,0.291498,-0.907420,...,-0.027501,-0.862050,0.558236,1.666989,-0.905734,0.154786,-2.474247,1.239481,-0.379410,-2.180697
1,-0.970972,0.599798,-0.201251,-1.238763,-1.670136,0.616025,-0.155953,0.164424,0.121662,-1.179901,...,-0.725462,-0.807263,0.070219,0.321139,-0.773257,1.175560,-0.989951,0.678261,-2.086529,-0.582212
2,-0.538919,-0.508422,0.382999,-0.467518,0.169280,-0.141333,-0.092597,-0.331947,0.176189,-0.317161,...,1.093093,0.608713,-0.088729,-0.630562,0.653220,0.388670,0.356475,-0.242725,0.937837,0.561793
3,-1.188363,0.384986,-0.372826,-1.264581,-1.667930,0.745715,-0.095575,0.340962,0.264248,-1.216824,...,-0.658865,-0.781946,0.139138,0.383648,-0.860902,1.321185,-1.168664,0.705715,-1.710215,-0.565700
4,-0.201625,0.175951,-0.409872,-1.394116,-1.645540,0.349398,0.195389,-0.137314,-0.765318,-0.390190,...,-0.006496,-0.019779,0.017412,0.497519,-1.146259,0.894981,-2.013336,0.346742,-1.932245,-1.398506
5,-0.724550,0.556368,-0.387734,-1.731118,-1.184754,0.391149,0.256508,-0.154196,0.585706,-1.295071,...,-0.564879,-0.939072,-0.398785,0.989990,-0.659343,0.762029,-1.028497,0.487470,-3.137863,-1.812966
6,-0.583464,-0.522868,0.409137,-0.545353,0.627849,-0.250539,-0.050176,-0.129662,0.121807,-0.253216,...,0.811983,0.497596,-0.318657,-0.469110,0.641332,0.456540,0.152262,-0.104693,0.734302,0.453637
7,-0.608990,0.636089,-0.208515,-1.404488,-1.082794,0.251478,0.180216,-0.564531,0.509240,-1.443204,...,-0.681035,-0.725354,-0.491961,0.858272,-0.485665,0.617391,-1.234728,0.531667,-2.990551,-1.672639
8,0.006160,0.110660,-0.246742,-1.407434,-1.855591,0.085666,0.167806,-0.257646,-0.772402,-0.553786,...,0.101223,0.100371,-0.029362,0.661633,-1.306451,0.879242,-1.593685,0.319464,-2.089104,-0.987625
9,0.482912,-0.606878,-0.564165,-0.930873,-0.997742,0.625884,-0.146066,0.144670,-0.749784,-0.849182,...,-0.943654,-0.437605,-0.031407,0.993969,0.082066,0.570855,-1.291363,0.531545,-2.027190,-0.623020


In [135]:
df['target'] = trainy

In [137]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-2.179760,0.270186,-2.484582,0.536236,-0.870110,-0.704696,-0.836423,0.391036,0.291498,-0.907420,...,-0.862050,0.558236,1.666989,-0.905734,0.154786,-2.474247,1.239481,-0.379410,-2.180697,poliana
1,-0.970972,0.599798,-0.201251,-1.238763,-1.670136,0.616025,-0.155953,0.164424,0.121662,-1.179901,...,-0.807263,0.070219,0.321139,-0.773257,1.175560,-0.989951,0.678261,-2.086529,-0.582212,poliana
2,-0.538919,-0.508422,0.382999,-0.467518,0.169280,-0.141333,-0.092597,-0.331947,0.176189,-0.317161,...,0.608713,-0.088729,-0.630562,0.653220,0.388670,0.356475,-0.242725,0.937837,0.561793,poliana
3,-1.188363,0.384986,-0.372826,-1.264581,-1.667930,0.745715,-0.095575,0.340962,0.264248,-1.216824,...,-0.781946,0.139138,0.383648,-0.860902,1.321185,-1.168664,0.705715,-1.710215,-0.565700,poliana
4,-0.201625,0.175951,-0.409872,-1.394116,-1.645540,0.349398,0.195389,-0.137314,-0.765318,-0.390190,...,-0.019779,0.017412,0.497519,-1.146259,0.894981,-2.013336,0.346742,-1.932245,-1.398506,poliana
5,-0.724550,0.556368,-0.387734,-1.731118,-1.184754,0.391149,0.256508,-0.154196,0.585706,-1.295071,...,-0.939072,-0.398785,0.989990,-0.659343,0.762029,-1.028497,0.487470,-3.137863,-1.812966,poliana
6,-0.583464,-0.522868,0.409137,-0.545353,0.627849,-0.250539,-0.050176,-0.129662,0.121807,-0.253216,...,0.497596,-0.318657,-0.469110,0.641332,0.456540,0.152262,-0.104693,0.734302,0.453637,poliana
7,-0.608990,0.636089,-0.208515,-1.404488,-1.082794,0.251478,0.180216,-0.564531,0.509240,-1.443204,...,-0.725354,-0.491961,0.858272,-0.485665,0.617391,-1.234728,0.531667,-2.990551,-1.672639,poliana
8,0.006160,0.110660,-0.246742,-1.407434,-1.855591,0.085666,0.167806,-0.257646,-0.772402,-0.553786,...,0.100371,-0.029362,0.661633,-1.306451,0.879242,-1.593685,0.319464,-2.089104,-0.987625,poliana
9,0.482912,-0.606878,-0.564165,-0.930873,-0.997742,0.625884,-0.146066,0.144670,-0.749784,-0.849182,...,-0.437605,-0.031407,0.993969,0.082066,0.570855,-1.291363,0.531545,-2.027190,-0.623020,poliana


In [138]:
df.to_csv('faces_validation.csv')